In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
# Tải dữ liệu
dataset = load_dataset("yelp_polarity")
print(dataset)

In [ ]:
# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# Tiền xử lý dữ liệu
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)

In [ ]:
small_train = dataset["train"].shuffle(seed=42).select(range(500))
small_eval = dataset["test"].shuffle(seed=42).select(range(200))

In [ ]:
# Khởi tạo mô hình
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Thiết lập tham số huấn luyện
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="yelp_review_classifier",
    eval_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    save_strategy="epoch",
    push_to_hub=False,
)

In [ ]:
# Quá trình huấn luyện mô hình
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
# Đánh giá mô hình sau khi huấn luyện và in ra kết quả
metrics = trainer.evaluate()
print(metrics)